# RR Lyrae

[RR Lyrae](https://en.wikipedia.org/wiki/RR_Lyrae) is the prototype of a class of variable stars with radial pulsations. The pulsation period is related to the stellar luminosity, and a second, lower frequency variation can be seen on longer timescales, known as the [Blazkho effect](https://en.wikipedia.org/wiki/Blazhko_effect). 

RR Lyrae itself was in the Kepler field, so let's load one Kepler Quarter of long-cadence observations, and inspect the periods:

In [ ]:
import astropy.units as u
from astropy.time import Time
from lightkurve import search_lightcurve

from lcviz import LCviz

# Download three quarters of long cadence observations from Kepler:
lc = search_lightcurve(
    target="RR Lyra", 
    mission="Kepler", 
    quarter=2, 
    cadence="long", 
    author="Kepler"
).download_all().stitch(
    # stitch together the quarters after normalizing each one:
    lambda x: x.normalize()
)

Now we'll start `LCviz`:

In [ ]:
lcviz = LCviz()

lcviz.load_data(lc)

# we'll make use of these plugins later:
eph = lcviz.plugins['Ephemeris']
plot_options = lcviz.plugins['Plot Options']
freq_analysis = lcviz.plugins['Frequency Analysis']
freq_analysis.auto_range = False
freq_analysis.xunit.selected = 'period'

lcviz.show()

Compute the peak power at periods near 13 hours using the `Frequency Analysis` plugin, and use the `Ephemeris` plugin to add a new viewer that's phase-folded at the pulsation period:

In [ ]:
# compute pulsation period:
freq_analysis.minimum = 0.1
freq_analysis.maximum = 1.0
ls = freq_analysis.periodogram
period_pulsation = 1 / ls.frequency[ls.power.argmax()]

# update Ephemeris:
eph.component.add_choice('pulsation')
eph.period = period_pulsation.to_value(u.d)
eph.t0 = 0.26

print(f"Pulsation period: {period_pulsation:.3f}")

Compute the peak power at periods near 36 days using the `Frequency Analysis` plugin, and add an `Ephemeris` component for the Blazkho effect:

In [ ]:
# compute Blazkho period:
freq_analysis.minimum = 20.0
freq_analysis.maximum = 55.0
ls = freq_analysis.periodogram
period_blazkho = 1 / ls.frequency[ls.power.argmax()]

# update Ephemeris:
eph.add_component('blazkho')
eph.period = period_blazkho.to_value(u.d)
eph.t0 = 23

print(f"Blazkho period: {period_blazkho:.3f}")

Color each point in the light curve in each viewer by the phase of the Blazkho effect, using the `Plot Options` plugin:

In [ ]:
for viewer in lcviz.app.get_viewer_reference_names():
    plot_options.viewer = viewer
    plot_options.layer = lcviz.app.data_collection[0].label
    plot_options.marker_color_mode = 'Linear'
    plot_options.marker_colormap = 'Viridis'
    plot_options._obj.marker_color_col_value = 'phase:blazkho'